<h1 style="color:blue;">📝 Problem Statement</h1>

In today’s world, information is everywhere. It has a ton of advantages, but it does not mean that it comes with zero cost. On the contrary, information overload is a serious problem as consumers can feel overwhelmed and either decide not to buy anything or do not take all options into account. To remedy this problem, a big food company for which you are working as data scientist wants to develop a recommendation system (RS) to provide better service to its customers.

RS can be defined as a system that makes personalized recommendations from a large range of different options by implicitly or explicitly eliciting the user’s preference for a product. RS benefits consumers as they reduce consumers' effort and search costs by making the decision process quick, easier, and painless.

As a data scientist at a big food company, you are asked to run item. Basically, user-based recommendation is a technique used to predict the items that a user might like based on ratings given to that item by the other users who have similar taste with that of the target user.


For this task, you are provided ‘recom.csv’ dataset. In this dataset, you have following columns:

+	Main_ID (Customer ID)
+	Transaction_ID
+	Date
+	Price
+	Code_Product
+	Amount (Order Amount)
+	ItemKey

Well, using this data, you are asked to generate a dataframe in which you recommend an item (or items) for each user.

Hints: <br>
1. Please note that there is no rating in the dataset, but, no worries, there is a way to solve this problem. 
2. If possible, please recommend multiple items per user.
3. You can consider creating pivot table with ‘Main_ID’, ‘ItemKey’, ‘Transaction_ID’.

<h4 style="color:red;">Import Libraries</h3>

In [208]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import colorama
from colorama import Fore, Style  # makes strings colored
from termcolor import colored
from termcolor import cprint

%matplotlib inline

<h4 style="color:red;">Load the dataset</h3>

In [209]:
df = pd.read_csv('recom.csv')

In [210]:
df.head(10)

,Unnamed: 0,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
0,0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
1,1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
2,2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
3,3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN
5,5,0e816717,654c3866,2022-10-21 20:53:45.037,72.0,49291.5,1.0,NaN
6,6,4cced191,87c4b548,2022-10-15 17:00:39.867,6.0,5012.0,1.0,5012.0
7,7,d93c0430,4e39a356,2022-10-21 10:50:28.717,33.5,49292.0,1.0,NaN
8,8,8ef0921e,c6a81d1e,2022-09-03 18:21:33.370,100.0,49292.0,3.0,NaN
9,9,2be09449,a2d55439,2022-10-28 12:23:00.727,90.0,5011.5,1.0,5011.5


<h4 style="color:red;">Simple function to handle heading display</h3>

In [211]:
#function handler to neat heading display
def heading(title):
  print(Fore.RED + '-'*50)
  print(Fore.BLUE + title.upper())
  print(Fore.RED + '-'*50)
  print(Fore.GREEN)

In [212]:
heading("Dropping the unnamed column")
df = df.drop(df.columns[0], axis=1)

--------------------------------------------------
DROPPING THE UNNAMED COLUMN
--------------------------------------------------



In [213]:
heading("Getting the shape o the dataset")
df.shape

--------------------------------------------------
GETTING THE SHAPE O THE DATASET
--------------------------------------------------



(50000, 7)

In [214]:
heading("Getting the dataset info summary")
df.info()

--------------------------------------------------
GETTING THE DATASET INFO SUMMARY
--------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Main_ID         50000 non-null  object 
 1   Transaction_ID  50000 non-null  object 
 2   Date            50000 non-null  object 
 3   Price           50000 non-null  float64
 4   Code_Product    50000 non-null  float64
 5   Amount          50000 non-null  float64
 6   ItemKey         28597 non-null  float64
dtypes: float64(4), object(3)
memory usage: 2.7+ MB


In [215]:
heading("Checking the missing value")
df.isna().sum()

--------------------------------------------------
CHECKING THE MISSING VALUE
--------------------------------------------------



Main_ID               0
Transaction_ID        0
Date                  0
Price                 0
Code_Product          0
Amount                0
ItemKey           21403
dtype: int64

In [216]:
heading("Get the count of equal value for Code_Product and ItemKey")
equalValue_count = (df['Code_Product'] == df['ItemKey']).sum()
equalValue_count

--------------------------------------------------
GET THE COUNT OF EQUAL VALUE FOR CODE_PRODUCT AND ITEMKEY
--------------------------------------------------



28597

In [217]:
heading("Get the count of missing value")
missing_ItemKey = df['ItemKey'].isna().sum()
missing_ItemKey

--------------------------------------------------
GET THE COUNT OF MISSING VALUE
--------------------------------------------------



21403

In [218]:
heading("Get the total count, should add up to 50000")
total_count = equalValue_count + missing_ItemKey
total_count

--------------------------------------------------
GET THE TOTAL COUNT, SHOULD ADD UP TO 50000
--------------------------------------------------



50000

In [219]:
heading("Dropping the ItemKey Column")
df.drop('ItemKey', axis=1, inplace=True)
df.head(10)

--------------------------------------------------
DROPPING THE ITEMKEY COLUMN
--------------------------------------------------



,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0
5,0e816717,654c3866,2022-10-21 20:53:45.037,72.0,49291.5,1.0
6,4cced191,87c4b548,2022-10-15 17:00:39.867,6.0,5012.0,1.0
7,d93c0430,4e39a356,2022-10-21 10:50:28.717,33.5,49292.0,1.0
8,8ef0921e,c6a81d1e,2022-09-03 18:21:33.370,100.0,49292.0,3.0
9,2be09449,a2d55439,2022-10-28 12:23:00.727,90.0,5011.5,1.0


In [220]:
heading("Converting the Code_Product and ItyemKey to string")
df["Code_Product"] = df["Code_Product"].astype(str)

--------------------------------------------------
CONVERTING THE CODE_PRODUCT AND ITYEMKEY TO STRING
--------------------------------------------------



In [221]:
heading("Checking the datatype after convertion")
df['Code_Product'].dtypes

--------------------------------------------------
CHECKING THE DATATYPE AFTER CONVERTION
--------------------------------------------------



dtype('O')

In [222]:

transaction_count = df.groupby('Main_ID')['Transaction_ID'].nunique()
multitransaction = transaction_count[transaction_count > 1]
multitransaction

Main_ID
00084856    2
000e98ee    2
0019e439    2
001ef3e0    2
002840dc    3
           ..
ffc818ff    4
ffcb01cb    2
ffe1ae80    2
ffe1fef8    3
fff9726b    2
Name: Transaction_ID, Length: 6666, dtype: int64

In [223]:
heading("Creating a Dataframe for all customers with multiple transactions")
user_with_multiple_transactions = multitransaction.index.tolist()
df_multitransactions = df[df['Main_ID'].isin(user_with_multiple_transactions)]
df_multitransactions.head(10)

--------------------------------------------------
CREATING A DATAFRAME FOR ALL CUSTOMERS WITH MULTIPLE TRANSACTIONS
--------------------------------------------------



,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0
5,0e816717,654c3866,2022-10-21 20:53:45.037,72.0,49291.5,1.0
7,d93c0430,4e39a356,2022-10-21 10:50:28.717,33.5,49292.0,1.0
10,45f3b09a,2d923b3a,2022-10-07 15:50:01.013,33.5,5025.0,1.0
11,eae34551,c22f7792,2022-09-12 17:28:35.257,164.0,10032.5,1.0
12,976e7a38,4157da21,2022-08-27 20:25:37.900,40.5,5009.0,1.0
14,34ed1014,8c413927,2022-11-28 20:43:22.280,9.5,40069.5,1.0
15,e023871c,882bff04,2022-09-09 18:54:52.577,53.5,45004.0,1.0
20,a8951bfb,e94b8071,2022-12-01 17:35:25.120,61.0,5025.0,1.0


In [224]:
heading("Grouping by Transaction_ID and get the total transaction price")
transaction_total = df.groupby('Transaction_ID')['Price'].sum().rename('Total_Transaction_Price')

--------------------------------------------------
GROUPING BY TRANSACTION_ID AND GET THE TOTAL TRANSACTION PRICE
--------------------------------------------------



In [225]:
heading("Merging the total transaction to the dataset")
df = df.merge(transaction_total, on='Transaction_ID')
df.head(20)

--------------------------------------------------
MERGING THE TOTAL TRANSACTION TO THE DATASET
--------------------------------------------------



,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,125.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,19.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,141.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,4.5
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,129.5
5,0e816717,654c3866,2022-10-21 20:53:45.037,72.0,49291.5,1.0,72.0
6,4cced191,87c4b548,2022-10-15 17:00:39.867,6.0,5012.0,1.0,6.0
7,d93c0430,4e39a356,2022-10-21 10:50:28.717,33.5,49292.0,1.0,33.5
8,8ef0921e,c6a81d1e,2022-09-03 18:21:33.370,100.0,49292.0,3.0,100.0
9,2be09449,a2d55439,2022-10-28 12:23:00.727,90.0,5011.5,1.0,90.0


<h4 style="color:red;">Taking some sample from the dataset</h3>

In [226]:
sample = df[df['Transaction_ID'] == 'c6a81d1e']
sample

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
8,8ef0921e,c6a81d1e,2022-09-03 18:21:33.370,100.0,49292.0,3.0,100.0


In [227]:
sample2 = df[df['Transaction_ID'] == '244fe6d8']
sample2

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,129.5


In [228]:
sample3 = df[df['Code_Product'] == 49291.5]
sample3

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price


In [229]:
sample4 = df[df['Transaction_ID'] == 'f6fe46f9']
sample4

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
49,58a375e6,f6fe46f9,2022-09-04 16:02:40.437,143.0,45001.0,1.0,286.0
50,58a375e6,f6fe46f9,2022-09-04 16:02:40.437,143.0,49291.5,1.0,286.0


In [230]:
mismatched_df = df[df["Price"] != df["Total_Transaction_Price"]]
mismatched_df

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
31,5a288e0d,f56648eb,2022-12-01 16:31:22.523,75.5,45001.0,1.0,226.5
32,5a288e0d,f56648eb,2022-12-01 16:31:22.523,75.5,49292.0,1.0,226.5
33,5a288e0d,f56648eb,2022-12-01 16:31:22.523,75.5,45004.0,1.0,226.5
49,58a375e6,f6fe46f9,2022-09-04 16:02:40.437,143.0,45001.0,1.0,286.0
50,58a375e6,f6fe46f9,2022-09-04 16:02:40.437,143.0,49291.5,1.0,286.0
...,...,...,...,...,...,...,...
48549,e0a9dd02,c72e1244,2022-09-16 17:32:52.677,115.0,45004.0,3.0,230.0
49052,b406b0b6,12dd7b95,2022-12-02 20:31:45.437,127.0,45002.5,1.0,254.0
49053,b406b0b6,12dd7b95,2022-12-02 20:31:45.437,127.0,45001.5,1.0,254.0
49744,b38c59f5,315045ef,2022-09-14 17:26:26.333,57.0,49291.5,1.0,114.0


In [231]:
sample5 = df[df['Transaction_ID'] == 'c72e1244']
sample5

,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,Total_Transaction_Price
48548,e0a9dd02,c72e1244,2022-09-16 17:32:52.677,115.0,48513.0,1.0,230.0
48549,e0a9dd02,c72e1244,2022-09-16 17:32:52.677,115.0,45004.0,3.0,230.0


<h4 style="color:red;">Observations</h3>

1. If Price is equal to Total Transaction Price then it is a single transaction only
2. Even if single transaction, the price is not necessarily the price of the item. It could be the price of the multiple same items bought 
   (ex. sample and sample2
3. Sample 3 shows same Product_ID, some are single transaction with single quantity of the product but shows different price.

<h4 style="color:red;">Create Pivot Table</h3>

In [232]:
heading("Create the Pivot Table")
pivot_table = pd.pivot_table(
    df,
    index='Main_ID',
    columns='Code_Product',
    values='Amount',
    aggfunc='count',
    fill_value=0  # replace NaN with 0
)

pivot_table

--------------------------------------------------
CREATE THE PIVOT TABLE
--------------------------------------------------



Code_Product,10000.5,10001.0,10001.5,10002.0,10003.0,10003.5,10004.0,10006.0,10013.0,10015.0,...,57035.5,60020.5,60047.5,60048.0,60048.5,60049.0,60049.5,60050.5,60051.0,60052.0
Main_ID,,,,,,,,,,,,,,,,,,,,,
00024de6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00084856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0008e848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00096930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000c66b7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff8b1c4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fff905d0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fff9726b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
from sklearn.metrics.pairwise import cosine_similarity
# heading("Transpose the pivot table to do item-based collaborative filtering")
# product_user_matrix = pivot_table2.T
# product_user_matrix

In [234]:
heading("Perform cosine similarity")
product_similarity = pd.DataFrame(cosine_similarity(product_user_matrix),index=product_user_matrix.index, columns=product_user_matrix.index)
product_similarity

--------------------------------------------------
PERFORM COSINE SIMILARITY
--------------------------------------------------



Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,1.000000,0.191281,0.132201,0.170314,0.0,0.013488,0.003334,0.066335,0.065775,0.043908,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.0,0.191281,1.000000,0.061417,0.133634,0.0,0.006876,0.000000,0.041405,0.105375,0.039498,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.5,0.132201,0.061417,1.000000,0.053683,0.0,0.015038,0.011152,0.013584,0.031428,0.025916,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.0,0.170314,0.133634,0.053683,1.000000,0.0,0.012020,0.000000,0.043429,0.046053,0.013810,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.5,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200045.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0
200046.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.091287,0.0,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0
200046.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [235]:
# def get_recom(item_id, similarity_matrix, top_n=5):
#     if item_id not in similarity_matrix:
#         return f"Product Code not found in data"
#     return similarity_matrix[item_id].sort_values(ascending=False)[1:top_n+1]

<h4 style="color:red;">Create Function to get recommendation</h3>

In [236]:
def get_recom(customer_id, item_id, pivot, sim_matrix, top_n=5):
    if customer_id not in pivot.index:
        return f"Customer {customer_id} not found in the pivot table"

    if item_id not in sim_matrix.index:
        return f"Item {item_id} not found in the similarity matrix."

    customerProfile = pivot.loc[customer_id]
    pastPurchased = customerProfile[customerProfile > 0].index.tolist()
    
    similar_items = sim_matrix[item_id].sort_values(ascending=False)
    recommendations = similar_items[~similar_items.index.isin(pastPurchased)]

    return recommendations.head(top_n)

<h4 style="color:red;">Testing the recommendation</h3>

In [237]:
customer_id = '00024de6'     
item_id = 200045.5    
recom = get_recom(customer_id, item_id, pivot_table, product_similarity)
print(recom)

Code_Product
200045.5    1.000000
200037.5    0.250000
35088.0     0.187317
200016.0    0.132453
165017.0    0.091287
Name: 200045.5, dtype: float64
